# Loading data

In [1]:
import pandas as pd
import numpy as np
pd.set_option('display.float_format', lambda x: '%.2f' % x)
import matplotlib.pyplot as plt
import math
from sklearn.metrics import mean_squared_error
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score


# DATA fusion
Code for fusion of data sensors without considering the imputation, plots and metrics calculus for the fusion and the singol sensors. 

RMSE, the F1 score and accuracy are calulated.

The RMSE score for each experiment is saved in the file 'RMSEvalues.csv' in the selected folder.

The Accuracy score for each experiment is saved in the file 'ACCvalues.csv' in the selected folder.

The F1 score for each experiment is saved in the file 'F1values.csv' in the selected folder.

To set desired experiments, filters, sensors and options, use next lines of code:

In [2]:
#Set option

#Set sensors: set sensors to consider by setting their variable to 'True'. For ITSC please use the configuration: range_sens='False', cam_w_yolo_FT='True', cam_d_yolo_FT='True', cam_w_yolo_v5='True', cam_d_yolo_v5='True'
range_sens='False' 
cam_w_yolo_FT='True'
cam_d_yolo_FT='True'
cam_w_yolo_v5='True'
cam_d_yolo_v5='True'

#Set experiments between 0 and 13. Multiple experiments can be set.
#set experiments of scene A or B. Scene A includes experiments from 0 to 5, while scene B from 6 to 11
scena='B'

if scena=='A':
    exp_list=[0,1,2,3,4,5]
elif scena=='B':
    exp_list=[6,7,8,9,10,11]

#set filters: between 'original','bright_1','bright_2','bright_3','dark_1','dark_2','dark_3','fog_1','fog_2','fog_3','rain_1','rain_2','rain_3'. Multiple filters can be set:
filter_list=['original','bright_1','bright_2','bright_3','dark_1','dark_2','dark_3','fog_1','fog_2','fog_3','rain_1','rain_2','rain_3']

#set folder name where the dataset is located
folder= "safe_road_data"

#set set_plot= 'True' if you want to see the plots:
set_plot= 'false'

print("exp_list="+ str(exp_list))
print("filter_list="+ str(filter_list))
print("folder="+ str(folder))

exp_list=[6, 7, 8, 9, 10, 11]
filter_list=['original', 'bright_1', 'bright_2', 'bright_3', 'dark_1', 'dark_2', 'dark_3', 'fog_1', 'fog_2', 'fog_3', 'rain_1', 'rain_2', 'rain_3']
folder=safe_road_data


# Danger function definition

In [3]:
# Distance component
def d_f(distance):
    if distance <= 0:
        return 0
    return 1 / math.log(distance + 0.6)

# Speed component
def d_s(speed):
    s_min = 0.05
    s_max = 0.5
    if speed <= s_min:
        return 0
    if speed >= s_max:
        return 1
    return (speed-s_min)/(s_max - s_min)

#Acceleration component
def d_a(acceleration):
    a_min = 1
    a_max = 10
    if acceleration <= - a_max:
        return -1
    elif -a_max < acceleration <= -a_min:
        return (acceleration+a_min)/(a_max - a_min)
    elif -a_min < acceleration <= +a_min:
        return 0
    elif a_min < acceleration <= +a_max:
        return (acceleration - a_min)/(a_max - a_min)
    elif acceleration > a_max:
        return 1
    return 0
    #if acceleration < 1 and acceleration > -1: # -1 < a < 1
        #return 1
    #elif acceleration < -1:
        #return 0.9
    #else:
        #return 1.1

# for plotting purposes returns just the distance component of the danger function
def danger_function_distance(row, sensor ,k1 = 1, k2 = 1, k3 = 1):
    distance = row[f'distance']+ 0.000001
    speed = row[f'speed']
    acceleration = row[f'acceleration']
    distance_factor = d_f(distance)
    speed_factor = d_s(speed)
    acceleration_factor = d_a(acceleration)
    return distance_factor

# for plotting purposes returns just the speed component of the danger function
def danger_function_speed(row, sensor ,k1 = 1, k2 = 1, k3 = 1):
    distance = row[f'distance']+ 0.000001
    speed = row[f'speed']
    acceleration = row[f'acceleration']
    distance_factor = d_f(distance)
    speed_factor = d_s(speed)
    acceleration_factor = d_a(acceleration) 
    return speed_factor

# for plotting purposes returns just the acceleration component of the danger function
def danger_function_acceleration(row, sensor ,k1 = 1, k2 = 1, k3 = 1):
    distance = row[f'distance']+ 0.000001
    speed = row[f'speed']
    acceleration = row[f'acceleration']
    distance_factor = d_f(distance)
    speed_factor = d_s(speed)
    acceleration_factor = d_a(acceleration)
    return acceleration_factor

# complete danger function, computes instant danger as combination of pre defined component
def danger_function_composed(row,sensor ,k1 = 0.1):
    distance = row[f'distance_{sensor}'] + 0.000001
    speed = row[f'speed_{sensor}']
    acceleration = row[f'acceleration_{sensor}']
    distance_factor = d_f(distance)
    speed_factor = d_s(speed)
    acceleration_factor = d_a(acceleration)
    return (speed_factor + k1 * acceleration_factor) * distance_factor

In [4]:

pd.set_option('display.max_rows', None)
pd.set_option('display.max_columns', None)
pd.set_option('display.width', None)

In [5]:
pd.reset_option('all')

/var/folders/kt/fxnfkb311578qq18r1dmj8tm0000gp/T/ipykernel_8652/2786130087.py:1: FutureWarning: data_manager option is deprecated and will be removed in a future version. Only the BlockManager will be available.
  pd.reset_option('all')
/var/folders/kt/fxnfkb311578qq18r1dmj8tm0000gp/T/ipykernel_8652/2786130087.py:1: FutureWarning: use_inf_as_na option is deprecated and will be removed in a future version. Convert inf values to NaN before operating instead.
  pd.reset_option('all')


In [6]:
F1values=pd.DataFrame()
RMSEvalues=pd.DataFrame()
ACCvalues=pd.DataFrame()

for filter in filter_list:
    exec(f"RMSE_{filter}= pd.DataFrame()")
    exec(f"F1_score_{filter}= pd.DataFrame()")
    exec(f"Accuracy_score_{filter}= pd.DataFrame()")
    
    exec(f"RMSE_{filter}_mean= pd.DataFrame()")
    exec(f"RMSE_{filter}_std= pd.DataFrame()")
    exec(f"Accuracy_score_{filter}_std= pd.DataFrame()")
    exec(f"F1_score_{filter}_mean= pd.DataFrame()")
    exec(f"F1_score_{filter}_std= pd.DataFrame()")
    exec(f"Accuracy_score_{filter}_std= pd.DataFrame()")
    #exec(f"F1values_{filter}=[]")



    for exp_nr in exp_list:
    

        #set folder name

        folder_name= f"{folder}/exp_{exp_nr}/pre_fusion_data"
        df_100 = pd.read_csv(f'{folder_name}/pre_fusion_data_{filter}.csv')
        if df_100.index[0]==0.00:
            df_100=df_100.drop(0.00, axis='index')
        #print(f'df_100 {exp_nr}', )
        df_100.index = df_100['Unnamed: 0']
        df_100.rename_axis('timestamp', inplace=True)
        df_100.drop(['Unnamed: 0'], axis=1, inplace=True)
        #print(f'ok2 {exp_nr}')
        # Distance Fusion
        ## Distance values fusion
        distance_fusion=pd.DataFrame()
        data=pd.DataFrame()
        data=df_100["wheelchair_data_FT"].copy(deep='True')
        if cam_w_yolo_FT=='True':
            distance_fusion = pd.concat([distance_fusion,df_100["distance_wheelchair_FT"]], axis=1)
            data=(data|df_100["wheelchair_data_FT"])
        if cam_w_yolo_v5=='True':
            distance_fusion = pd.concat([distance_fusion,df_100["distance_wheelchair_v5"]], axis=1)  
            data=(data|df_100["wheelchair_data_v5"])  
        if cam_d_yolo_FT=='True':
            distance_fusion = pd.concat([distance_fusion,df_100["distance_drone_FT"]], axis=1)    
            data=(data|df_100["drone_data_FT"])
        if cam_d_yolo_v5=='True':
            distance_fusion = pd.concat([distance_fusion,df_100["distance_drone_v5"]], axis=1)    
            data=(data|df_100["drone_data_v5"])
        if range_sens=='True':
            distance_fusion = pd.concat([distance_fusion,df_100["distance_range"]], axis=1)    
            data=(data|df_100["range_data"])
            
        df_100['distance_fusion'] = distance_fusion.mean(axis=1)


        # Distance Fusion

      
        data=data.astype(int)
        dist = df_100['distance_fusion']*data
        dist = dist.replace(0, np.nan)
        

        #stamps = df_100.index*(df_100['wheelchair_data'] | df_100['drone_data'] | df_100['range_data']).astype(int)
        stamps = df_100.index*data
        stamps = stamps.replace(0, np.nan)

        ## Speed values fusion
        df_speed = pd.DataFrame(dist, index=stamps, columns=['distance_fusion'])
        #df_speed=pd.DataFrame()
        df_speed = df_speed.dropna()        
        df_speed['speed_fusion'] = np.insert(- np.diff(df_speed['distance_fusion'])/np.diff(df_speed.index), 0, 0)
        max_speed = 3.5
        min_speed = 0 # We assume the obstacle is not going backwards
        df_speed['speed_fusion'] = np.clip(df_speed['speed_fusion'], a_min=min_speed, a_max=max_speed)
        df_100['speed_fusion'] = df_speed['speed_fusion'].copy(deep='True')
        df_100['speed_fusion'] = df_100['speed_fusion'].ffill()

        ## Acceleration values fusion
        df_speed['acceleration_fusion'] = np.insert(- np.diff(df_speed['speed_fusion'])/np.diff(df_speed.index), 0, 0)
        df_100['acceleration_fusion'] = df_speed['acceleration_fusion']
        df_100['acceleration_fusion'] = df_100['acceleration_fusion'].ffill()

        df_100['speed_fusion'] = df_100['speed_fusion'].fillna(0)
        df_100['acceleration_fusion'] = df_100['acceleration_fusion'].fillna(0)
        

        ## Danger function application on distance fusion and tracker danger computation
        df_100['danger_fusion'] = df_100.apply(lambda row: danger_function_composed(row, 'fusion'), axis=1)
        df_100['danger_tracker'] = df_100.apply(lambda row: danger_function_composed(row, 'tracker'), axis=1)
        df_100['danger_tracker'] =df_100['danger_tracker'].fillna(0)
        df_dist_noroll=df_100['danger_tracker'].copy(deep='True')
        #df_100['danger_fusion'] = df_100['danger_fusion'].rolling(window=20).mean()
        df_100['danger_fusion'] = df_100['danger_fusion'].fillna(0)
        
        if set_plot=='True':
            #plot start---------------------
            fig, ax = plt.subplots(3,1)
            # Index 4 Axes arrays in 4 subplots within 1 Figure: 
            ax[0].title.set_text(f'Distance comparison exp {exp_nr} with filter {filter}')
            ax[0].plot(df_100.index, df_100['distance_tracker'], label='track')
            ax[0].plot(df_100.index, df_100['distance_fusion'], label='fus')

            if cam_d_yolo_FT=='True':
                ax[0].plot(df_100.index, df_100['distance_drone_FT'], label='d_FT')
            if cam_d_yolo_v5=='True':
                ax[0].plot(df_100.index, df_100['distance_drone_v5'], label='d_v5')#,marker='o', linestyle=' ')
            if cam_w_yolo_FT=='True':
                ax[0].plot(df_100.index, df_100['distance_wheelchair_FT'], label='w_FT')
            if cam_w_yolo_v5=='True':
                ax[0].plot(df_100.index, df_100['distance_wheelchair_v5'], label='w_v5')
            ax[0].set_ylabel('')
            ax[0].set_xlabel('Time [s]')
            ax[0].grid()
            ax[0].legend(loc='lower left')
        

            #plt.title(f'Danger comparison with smoothing exp {exp_nr}')
            ax[1].title.set_text('Danger comparison')
            ax[1].plot(df_100.index, df_100['danger_tracker'], label='tracker')
            #ax[1].plot(df_100.index,df_dist_noroll, label='fusion')
            ax[1].plot(df_100.index, df_100['danger_fusion'], label='fusion smooth')
            if cam_d_yolo_FT=='True':
                ax[1].plot(df_100.index, df_100['danger_drone_FT'], label='d_FT')
            if cam_d_yolo_v5=='True':
                ax[1].plot(df_100.index, df_100['danger_drone_v5'], label='d_v5')#,marker='o', linestyle=' ')
            if cam_w_yolo_FT=='True':
                ax[1].plot(df_100.index, df_100['danger_wheelchair_FT'], label='w_FT')
            if cam_w_yolo_v5=='True':
                ax[1].plot(df_100.index, df_100['danger_wheelchair_v5'], label='w_v5')
            ax[1].plot(df_100.index, np.ones(len(df_100)), label='fusion')
        
            ax[1].set_ylabel('')
            ax[1].set_xlabel('Time [s]')
            ax[1].grid()
            #ax[1].legend(loc="upper right")
            #plot end---------------------_

        #to evaluate from first available data of each sensor 
        first_valid_index = df_100['danger_fusion'].first_valid_index()

        
        #viene rifatto meglio dopo (non considerare)
        #truth = df_100['danger_tracker'].loc[first_valid_index:] < 1
        #decision = df_100['danger_fusion'].loc[first_valid_index:] < 1 #memory of the semaphore

        ## Decision (threshold after DF)
        ### Decision loop
        ### This loop achieves a binary cross/no-cross decision for distance (low-level) fusion

        #considerare questo:
        decision = []
        memory = []
        for i, item in enumerate(df_100['danger_fusion']):
            if item is np.nan:
                item = 1
            if i<10:
                decision.append(0)
                memory.append(0)
            else:
                if sum(memory) + item >= 1:
                    decision.append(1)
                    if item >= 1:
                        memory.append(1)
                    else:
                        memory.append(0)
                    del memory[0]
                else:
                    decision.append(0)
                    memory.append(0)
                    del memory[0]

        #df_100.index = df_100.index - df_100.index[0] #?? cosa succede se lo tolgo?

        #df_100.to_csv('distance_fusion.csv')

        # cast list to pandas format
        decision = pd.Series(decision, index=df_100.index)
        # Make tracker decision binary, cross/no-cross
        truth = df_100['danger_tracker']>1
        # plot real danger function and real decision


        if set_plot=='True':
            #plot start---------------------_
            
            ax[2].title.set_text('Decision comparison')
            ax[2].plot(df_100.index, df_100['danger_tracker'], label='tracker')
            ax[2].plot(df_100.index, df_100['danger_fusion'], label='fusion')
            ax[2].plot(df_100.index, np.ones(len(df_100)), label='fusion')
            ax[2].scatter(df_100.index, truth, label='truth')
            ax[2].scatter(df_100.index, decision, label='decision')
            ax[2].set_ylabel('')
            ax[2].set_xlabel('Time [s]')
            ax[2].grid()
            ax[2].legend(loc="upper right")
            #plot end---------------------_

            plt.show()

        ## RMSE for distance fusion
        if first_valid_index!= None:
            #y_actual = df_100['danger_tracker'].loc[first_valid_index:]
            #y_pred = df_100['danger_fusion'].loc[first_valid_index:]
            y_actual= df_100['danger_tracker']
            y_pred = df_100['danger_fusion']
            #y_pred_max=df_100['danger_fusion'].max()
            #y_pred_min=df_100['danger_fusion'].min()
            #y_pred_min=0
           ###print(y_actual,y_pred)
            MSE = mean_squared_error(y_actual, y_pred)
            rmse = np.sqrt(MSE)
            
            ### Decision metrics
            truth = df_100['danger_tracker']>1
            decision = df_100['danger_fusion']>1
            accuracy = accuracy_score(truth, decision)
            precision = precision_score(truth, decision)
            recall = recall_score(truth, decision)
            f1 = f1_score(truth, decision)
            
        else:
            rmse= None
            f1 = None
            accuracy = None

        RMSE=pd.DataFrame()
        F1_score=pd.DataFrame()
        Accuracy_score=pd.DataFrame()

        if 'fusion' not in RMSE:
            RMSE.insert(0, "fusion", [rmse])
        if 'fusion' not in F1_score:
            F1_score.insert(0, "fusion", [f1])
        if 'fusion' not in Accuracy_score:
            Accuracy_score.insert(0, "fusion", [accuracy])
        

        #accuracy, precision, recall, f1        
        #Metrics of single sensors
        ## For SAFECOMP: RMSE for individual sensors
        if range_sens=='True':
            ### RMSE range
            df_100['danger_range'] = df_100.apply(danger_function_composed, args=('range'), axis=1)
            first_valid_index_range = df_100['danger_range'].first_valid_index()
            if first_valid_index_range!= None:
                y_actual = df_100['danger_tracker']#.loc[first_valid_index_range:]
                y_pred = df_100['danger_range']#.loc[first_valid_index_range:]
                #y_pred_max=df_100['danger_range'].max()
                #y_pred_min=df_100['danger_range'].min()
                #y_pred_min=0
               ##print(y_actual,y_pred)
                MSE = mean_squared_error(y_actual, y_pred)
                rmse = np.sqrt(MSE)
               
                # Calculate F1-score
                ### This loop achieves a binary cross/no-cross decision for distance (low-level) fusion
                #considerare questo:
                decision = []
                memory = []
                for i, item in enumerate(df_100['danger_range']):
                    if item is np.nan:
                        item = 1
                    if i<10:
                        decision.append(0)
                        memory.append(0)
                    else:
                        if sum(memory) + item >= 1:
                            decision.append(1)
                            if item >= 1:
                                memory.append(1)
                            else:
                                memory.append(0)
                            del memory[0]
                        else:
                            decision.append(0)
                            memory.append(0)
                            del memory[0]

                #df_100.index = df_100.index - df_100.index[0] #?? cosa succede se lo tolgo?
                # cast list to pandas format
                decision = pd.Series(decision, index=df_100.index)
                decision = df_100['danger_range']>1
                f1 = f1_score(truth, decision)
                accuracy = accuracy_score(truth, decision)
            else:
                rmse= None
                f1 = None
                accuracy = None

            if 'range_sens' not in RMSE:
                RMSE.insert(0, "range_sens", [rmse])
            if 'range_sens' not in F1_score:
                F1_score.insert(0, "range_sens", [f1])
            if 'range_sens' not in Accuracy_score:
                Accuracy_score.insert(0, "range_sens", [accuracy])



        if cam_w_yolo_FT=='True':
            ### RMSE wheelchair FT
            df_100['danger_wheelchair_FT'] = df_100.apply(danger_function_composed, args=('wheelchair_FT',), axis=1)

            first_valid_index_wheelchair = df_100['danger_wheelchair_FT'].first_valid_index()
            if first_valid_index_wheelchair!= None:

                y_actual = df_100['danger_tracker']#.loc[first_valid_index_wheelchair:]
                y_pred = df_100['danger_wheelchair_FT']#.loc[first_valid_index_wheelchair:]
                #y_pred_max=df_100['danger_wheelchair_FT'].max()
                #y_pred_min=df_100['danger_wheelchair_FT'].min()
                #y_pred_min=0
               ##print(y_actual,y_pred)
                MSE = mean_squared_error(y_actual, y_pred)
                #rmse = np.sqrt(MSE)/(y_pred_max-y_pred_min)
                rmse = np.sqrt(MSE)

                ### This loop achieves a binary cross/no-cross decision for distance (low-level) fusion
                #considerare questo:
                decision = []
                memory = []
                for i, item in enumerate(df_100['danger_wheelchair_FT']):
                    if item is np.nan:
                        item = 1
                    if i<10:
                        decision.append(0)
                        memory.append(0)
                    else:
                        if sum(memory) + item >= 1:
                            decision.append(1)
                            if item >= 1:
                                memory.append(1)
                            else:
                                memory.append(0)
                            del memory[0]
                        else:
                            decision.append(0)
                            memory.append(0)
                            del memory[0]

                #df_100.index = df_100.index - df_100.index[0] #?? cosa succede se lo tolgo?
                #df_100.to_csv('distance_fusion.csv')
                # cast list to pandas format
                decision = pd.Series(decision, index=df_100.index)
                decision = df_100['danger_wheelchair_FT']>1
                f1 = f1_score(truth, decision)
                accuracy = accuracy_score(truth, decision)
            else:
                rmse= None
                f1 = None
                accuracy = None
    
            if 'cam_w_yolo_FT' not in RMSE:
                RMSE.insert(0, "cam_w_yolo_FT", [rmse])
            if 'cam_w_yolo_FT' not in F1_score:
                F1_score.insert(0, "cam_w_yolo_FT", [f1])
            if 'cam_w_yolo_FT' not in Accuracy_score:
                Accuracy_score.insert(0, "cam_w_yolo_FT", [accuracy])


        if cam_w_yolo_v5=='True':
            ### RMSE wheelchair v5
            df_100['danger_wheelchair_v5'] = df_100.apply(danger_function_composed, args=('wheelchair_v5',), axis=1)

            first_valid_index_wheelchair = df_100['danger_wheelchair_v5'].first_valid_index()
            if first_valid_index_wheelchair!= None:
                y_actual = df_100['danger_tracker']#.loc[first_valid_index_wheelchair:]
                y_pred = df_100['danger_wheelchair_v5']#.loc[first_valid_index_wheelchair:]
                #y_pred_max=df_100['danger_wheelchair_v5'].max()
                #y_pred_min=df_100['danger_wheelchair_v5'].min()
                #y_pred_min=0
               #print(y_actual,y_pred)
                MSE = mean_squared_error(y_actual, y_pred)
                #rmse = np.sqrt(MSE)/(y_pred_max-y_pred_min)
                rmse = np.sqrt(MSE)


                ### This loop achieves a binary cross/no-cross decision for distance (low-level) fusion
                #considerare questo:
                decision = []
                memory = []
                for i, item in enumerate(df_100['danger_wheelchair_v5']):
                    if item is np.nan:
                        item = 1
                    if i<10:
                        decision.append(0)
                        memory.append(0)
                    else:
                        if sum(memory) + item >= 1:
                            decision.append(1)
                            if item >= 1:
                                memory.append(1)
                            else:
                                memory.append(0)
                            del memory[0]
                        else:
                            decision.append(0)
                            memory.append(0)
                            del memory[0]

                #df_100.index = df_100.index - df_100.index[0] #?? cosa succede se lo tolgo?
                #df_100.to_csv('distance_fusion.csv')
                # cast list to pandas format
                decision = pd.Series(decision, index=df_100.index)
                decision = df_100['danger_wheelchair_v5']>1
                f1 = f1_score(truth, decision)
                accuracy = accuracy_score(truth, decision)
            else:
                rmse= None
                f1 = None
                accuracy = None
    
            if 'cam_w_yolo_v5' not in RMSE:
                RMSE.insert(0, "cam_w_yolo_v5", [rmse])
            if 'cam_w_yolo_v5' not in F1_score:
                F1_score.insert(0, "cam_w_yolo_v5", [f1])
            if 'cam_w_yolo_v5' not in Accuracy_score:
                Accuracy_score.insert(0, "cam_w_yolo_v5", [accuracy])



        if cam_d_yolo_FT=='True':  
            ### RMSE drone FT
            df_100['danger_drone_FT'] = df_100.apply(danger_function_composed, args=('drone_FT',), axis=1)

            first_valid_index_drone = df_100['danger_drone_FT'].first_valid_index()
            if first_valid_index_drone!= None:
                y_actual = df_100['danger_tracker']#.loc[first_valid_index_drone:]
                y_pred = df_100['danger_drone_FT']#.loc[first_valid_index_drone:]
                #y_pred_max=df_100['danger_drone_FT'].max()
                #y_pred_min=df_100['danger_drone_FT'].min()
                #y_pred_min=0
               #print(y_actual,y_pred)
                MSE = mean_squared_error(y_actual, y_pred)
                rmse = np.sqrt(MSE)

                
                ### This loop achieves a binary cross/no-cross decision for distance (low-level) fusion
                #considerare questo:
                decision = []
                memory = []
                for i, item in enumerate(df_100['danger_drone_FT']):
                    if item is np.nan:
                        item = 1
                    if i<10:
                        decision.append(0)
                        memory.append(0)
                    else:
                        if sum(memory) + item >= 1:
                            decision.append(1)
                            if item >= 1:
                                memory.append(1)
                            else:
                                memory.append(0)
                            del memory[0]
                        else:
                            decision.append(0)
                            memory.append(0)
                            del memory[0]

                #df_100.index = df_100.index - df_100.index[0] #?? cosa succede se lo tolgo?
                #df_100.to_csv('distance_fusion.csv')
                # cast list to pandas format
                decision = pd.Series(decision, index=df_100.index)
                decision = df_100['danger_drone_FT']>1
                f1 = f1_score(truth, decision)
                accuracy = accuracy_score(truth, decision)
            else:
                rmse= None
                f1 = None
                accuracy = None

            if 'cam_d_yolo_FT' not in RMSE:
                RMSE.insert(0, "cam_d_yolo_FT", [rmse])
            if 'cam_d_yolo_FT' not in F1_score:
                F1_score.insert(0, "cam_d_yolo_FT", [f1])
            if 'cam_d_yolo_FT' not in Accuracy_score:
                Accuracy_score.insert(0, "cam_d_yolo_FT", [accuracy])


        if cam_d_yolo_v5=='True':  
            ### RMSE drone v5
            df_100['danger_drone_v5'] = df_100.apply(danger_function_composed, args=('drone_v5',), axis=1)

            first_valid_index_drone = df_100['danger_drone_v5'].first_valid_index()
            if first_valid_index_drone!= None:
                y_actual = df_100['danger_tracker']#.loc[first_valid_index_drone:]
                y_pred = df_100['danger_drone_v5']#.loc[first_valid_index_drone:]
                #y_pred_max=df_100['danger_drone_v5'].max()
                #y_pred_min=df_100['danger_drone_v5'].min()
                #y_pred_min=0
                #print(y_actual,y_pred)
                MSE = mean_squared_error(y_actual, y_pred)
                rmse = np.sqrt(MSE)


                ### This loop achieves a binary cross/no-cross decision for distance (low-level) fusion
                #considerare questo:
                decision = []
                memory = []
                for i, item in enumerate(df_100['danger_drone_v5']):
                    if item is np.nan:
                        item = 1
                    if i<10:
                        decision.append(0)
                        memory.append(0)
                    else:
                        if sum(memory) + item >= 1:
                            decision.append(1)
                            if item >= 1:
                                memory.append(1)
                            else:
                                memory.append(0)
                            del memory[0]
                        else:
                            decision.append(0)
                            memory.append(0)
                            del memory[0]

                #df_100.index = df_100.index - df_100.index[0] #?? cosa succede se lo tolgo?
                #df_100.to_csv('distance_fusion.csv')
                # cast list to pandas format
                decision = pd.Series(decision, index=df_100.index)
                decision = df_100['danger_drone_v5']>1
                f1 = f1_score(truth, decision)
                accuracy = accuracy_score(truth, decision)
            else:
                rmse= None
                f1 = None
                accuracy = None

            if 'cam_d_yolo_v5' not in RMSE:
                RMSE.insert(0, "cam_d_yolo_v5", [rmse])
            if 'cam_d_yolo_v5' not in F1_score:
                F1_score.insert(0, "cam_d_yolo_v5", [f1])
            if 'cam_d_yolo_v5' not in Accuracy_score:
                Accuracy_score.insert(0, "cam_d_yolo_v5", [accuracy])

        #print('RMSE='+str(RMSE))
        #print('F1_score='+str(F1_score))

        exec(f"RMSE_{filter}= pd.concat([RMSE_{filter}, RMSE], ignore_index=True)")

        exec(f"F1_score_{filter}= pd.concat([F1_score_{filter}, F1_score], ignore_index=True)")
        exec(f"Accuracy_score_{filter}= pd.concat([Accuracy_score_{filter}, Accuracy_score], ignore_index=True)")

        #exec(f"print('RMSE_{filter}='+str(RMSE_{filter}))")
        #exec(f"print('F1_score_{filter}='+str(F1_score_{filter}))")
        #exec(f"print('Accuracy_score_{filter}='+str(Accuracy_score_{filter}))")
    
    exec(f"RMSE_{filter}_mean= RMSE_{filter}.mean()")
    exec(f"RMSE_{filter}_std=  RMSE_{filter}.std()")
    exec(f"F1_score_{filter}_mean= F1_score_{filter}.mean()*100")
    exec(f"F1_score_{filter}_std= F1_score_{filter}.std()*100")
    exec(f"Accuracy_score_{filter}_mean= Accuracy_score_{filter}.mean()*100")
    exec(f"Accuracy_score_{filter}_std= Accuracy_score_{filter}.std()*100")

    #F1values for each experiment
    exec(f"F1values= pd.concat([F1values, F1_score_{filter}],axis=1,ignore_index=False)")
    exec(f"str_cam_d_yolo_FT='cam_d_yolo_FT_{filter}' ")
    exec(f"str_cam_d_yolo_v5='cam_d_yolo_v5_{filter}' ")
    exec(f"str_cam_w_yolo_FT='cam_w_yolo_FT_{filter}' ")
    exec(f"str_cam_w_yolo_v5='cam_w_yolo_v5_{filter}' ")
    exec(f"str_fusion_old='fusion_old_{filter}' ")

    F1values=F1values.rename(columns={'cam_d_yolo_v5': str_cam_d_yolo_v5})
    F1values=F1values.rename(columns={'cam_d_yolo_FT': str_cam_d_yolo_FT})
    F1values=F1values.rename(columns={'cam_w_yolo_v5': str_cam_w_yolo_v5})
    F1values=F1values.rename(columns={'cam_w_yolo_FT': str_cam_w_yolo_FT})
    F1values=F1values.rename(columns={'fusion': str_fusion_old})
    
    #RMSEvalues for each experiment
    exec(f"RMSEvalues= pd.concat([RMSEvalues, RMSE_{filter}],axis=1,ignore_index=False)")
    exec(f"str_cam_d_yolo_FT='cam_d_yolo_FT_{filter}' ")
    exec(f"str_cam_d_yolo_v5='cam_d_yolo_v5_{filter}' ")
    exec(f"str_cam_w_yolo_FT='cam_w_yolo_FT_{filter}' ")
    exec(f"str_cam_w_yolo_v5='cam_w_yolo_v5_{filter}' ")
    exec(f"str_fusion_old='fusion_old_{filter}' ")

    RMSEvalues=RMSEvalues.rename(columns={'cam_d_yolo_v5': str_cam_d_yolo_v5})
    RMSEvalues=RMSEvalues.rename(columns={'cam_d_yolo_FT': str_cam_d_yolo_FT})
    RMSEvalues=RMSEvalues.rename(columns={'cam_w_yolo_v5': str_cam_w_yolo_v5})
    RMSEvalues=RMSEvalues.rename(columns={'cam_w_yolo_FT': str_cam_w_yolo_FT})
    RMSEvalues=RMSEvalues.rename(columns={'fusion': str_fusion_old})

    #Accuracy values for each experiment
    exec(f"ACCvalues= pd.concat([ACCvalues, Accuracy_score_{filter}],axis=1,ignore_index=False)")
    exec(f"str_cam_d_yolo_FT='cam_d_yolo_FT_{filter}' ")
    exec(f"str_cam_d_yolo_v5='cam_d_yolo_v5_{filter}' ")
    exec(f"str_cam_w_yolo_FT='cam_w_yolo_FT_{filter}' ")
    exec(f"str_cam_w_yolo_v5='cam_w_yolo_v5_{filter}' ")
    exec(f"str_fusion_old='fusion_old_{filter}' ")

    ACCvalues=ACCvalues.rename(columns={'cam_d_yolo_v5': str_cam_d_yolo_v5})
    ACCvalues=ACCvalues.rename(columns={'cam_d_yolo_FT': str_cam_d_yolo_FT})
    ACCvalues=ACCvalues.rename(columns={'cam_w_yolo_v5': str_cam_w_yolo_v5})
    ACCvalues=ACCvalues.rename(columns={'cam_w_yolo_FT': str_cam_w_yolo_FT})
    ACCvalues=ACCvalues.rename(columns={'fusion': str_fusion_old})



folder_name='safe_road_data/'
F1values.to_csv(f'{folder_name}/F1values.csv')
RMSEvalues.to_csv(f'{folder_name}/RMSEvalues.csv')
ACCvalues.to_csv(f'{folder_name}/ACCvalues.csv')




/Library/Frameworks/Python.framework/Versions/3.12/lib/python3.12/site-packages/sklearn/metrics/_classification.py:1509: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/Library/Frameworks/Python.framework/Versions/3.12/lib/python3.12/site-packages/sklearn/metrics/_classification.py:1509: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/Library/Frameworks/Python.framework/Versions/3.12/lib/python3.12/site-packages/sklearn/metrics/_classification.py:1509: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modif

## Metrics: RMSE, Accuracy and F1 score

### Mean and standard deviation

In [7]:
pd.set_option('display.float_format', lambda x: '%.2f' % x)
RMSE_original_mean.name='RMSE_original_mean'
RMSE_original_std.name='RMSE_original_std'
Accuracy_score_original_mean.name='Accuracy_score_original_mean'
Accuracy_score_original_std.name='Accuracy_score_original_std'
F1_score_original_mean.name='F1_score_original_mean'
F1_score_original_std.name='F1_score_original_std'

pd.concat([RMSE_original_mean,RMSE_original_std, Accuracy_score_original_mean, Accuracy_score_original_std, F1_score_original_mean, F1_score_original_std], axis=1)



,RMSE_original_mean,RMSE_original_std,Accuracy_score_original_mean,Accuracy_score_original_std,F1_score_original_mean,F1_score_original_std
cam_d_yolo_v5,1.26,0.32,67.95,11.02,41.58,14.88
cam_d_yolo_FT,1.04,0.30,63.51,15.39,17.50,3.73
cam_w_yolo_v5,1.08,0.22,68.49,11.52,35.71,11.40
cam_w_yolo_FT,1.00,0.30,70.89,11.13,36.40,16.52
fusion,0.88,0.27,64.42,15.08,18.45,3.28


### Values for single esperiment

In [8]:
F1values

,cam_d_yolo_v5_original,cam_d_yolo_FT_original,cam_w_yolo_v5_original,cam_w_yolo_FT_original,fusion_old_original,cam_d_yolo_v5_bright_1,cam_d_yolo_FT_bright_1,cam_w_yolo_v5_bright_1,cam_w_yolo_FT_bright_1,fusion_old_bright_1,...,cam_d_yolo_v5_rain_2,cam_d_yolo_FT_rain_2,cam_w_yolo_v5_rain_2,cam_w_yolo_FT_rain_2,fusion_old_rain_2,cam_d_yolo_v5_rain_3,cam_d_yolo_FT_rain_3,cam_w_yolo_v5_rain_3,cam_w_yolo_FT_rain_3,fusion_old_rain_3
0,0.26,0.21,0.18,0.04,0.16,0.22,0.14,0.14,0.04,0.10,...,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00
1,0.63,0.18,0.29,0.48,0.22,0.64,0.22,0.28,0.57,0.19,...,0.00,0.10,0.00,0.30,0.00,0.00,0.00,0.00,0.00,0.00
2,0.48,0.15,0.47,0.36,0.17,0.58,0.36,0.36,0.41,0.21,...,0.00,0.06,0.00,0.22,0.00,0.00,0.00,0.00,0.00,0.00
3,0.52,0.21,0.48,0.41,0.22,0.61,0.43,0.39,0.39,0.26,...,0.00,0.13,0.00,0.22,0.00,0.00,0.00,0.00,0.00,0.00
4,0.33,0.18,0.32,0.40,0.18,0.40,0.45,0.24,0.49,0.34,...,0.00,0.06,0.00,0.24,0.00,0.00,0.00,0.00,0.03,0.00
5,0.27,0.12,0.40,0.49,0.14,0.43,0.49,0.28,0.63,0.25,...,0.00,0.19,0.00,0.24,0.00,0.00,0.00,0.00,0.00,0.00


In [9]:
RMSEvalues

,cam_d_yolo_v5_original,cam_d_yolo_FT_original,cam_w_yolo_v5_original,cam_w_yolo_FT_original,fusion_old_original,cam_d_yolo_v5_bright_1,cam_d_yolo_FT_bright_1,cam_w_yolo_v5_bright_1,cam_w_yolo_FT_bright_1,fusion_old_bright_1,...,cam_d_yolo_v5_rain_2,cam_d_yolo_FT_rain_2,cam_w_yolo_v5_rain_2,cam_w_yolo_FT_rain_2,fusion_old_rain_2,cam_d_yolo_v5_rain_3,cam_d_yolo_FT_rain_3,cam_w_yolo_v5_rain_3,cam_w_yolo_FT_rain_3,fusion_old_rain_3
0,0.85,0.64,0.77,0.65,0.53,0.87,0.62,0.82,0.63,0.50,...,0.67,0.66,0.67,0.66,0.58,0.67,0.67,0.67,0.66,0.64
1,1.12,1.39,1.39,1.32,1.21,1.14,1.35,1.39,1.26,1.21,...,1.51,1.46,1.51,1.39,1.38,1.51,1.51,1.51,1.51,1.41
2,1.31,1.37,1.26,1.35,1.15,1.18,1.30,1.30,1.32,1.16,...,1.49,1.47,1.49,1.40,1.33,1.49,1.48,1.49,1.49,1.24
3,1.04,1.09,1.00,1.09,0.93,0.98,0.98,1.04,1.07,0.95,...,1.22,1.18,1.22,1.15,1.00,1.22,1.22,1.22,1.22,1.18
4,1.68,0.84,1.00,0.81,0.68,1.57,0.70,1.03,0.79,0.65,...,0.90,0.89,0.90,0.87,0.83,0.90,0.90,0.90,0.90,0.89
5,1.59,0.92,1.03,0.79,0.75,1.58,0.79,1.22,0.74,0.67,...,0.98,0.91,0.98,0.89,0.87,0.98,0.98,0.98,0.98,0.80


In [10]:
ACCvalues

,cam_d_yolo_v5_original,cam_d_yolo_FT_original,cam_w_yolo_v5_original,cam_w_yolo_FT_original,fusion_old_original,cam_d_yolo_v5_bright_1,cam_d_yolo_FT_bright_1,cam_w_yolo_v5_bright_1,cam_w_yolo_FT_bright_1,fusion_old_bright_1,...,cam_d_yolo_v5_rain_2,cam_d_yolo_FT_rain_2,cam_w_yolo_v5_rain_2,cam_w_yolo_FT_rain_2,fusion_old_rain_2,cam_d_yolo_v5_rain_3,cam_d_yolo_FT_rain_3,cam_w_yolo_v5_rain_3,cam_w_yolo_FT_rain_3,fusion_old_rain_3
0,0.86,0.91,0.87,0.87,0.91,0.82,0.87,0.88,0.85,0.89,...,0.95,0.93,0.95,0.91,0.95,0.95,0.95,0.95,0.92,0.95
1,0.70,0.50,0.54,0.62,0.51,0.70,0.51,0.53,0.66,0.50,...,0.44,0.47,0.44,0.54,0.44,0.44,0.44,0.44,0.44,0.44
2,0.61,0.48,0.61,0.56,0.49,0.66,0.56,0.56,0.58,0.50,...,0.43,0.45,0.43,0.50,0.43,0.43,0.43,0.43,0.43,0.43
3,0.75,0.65,0.73,0.71,0.66,0.78,0.71,0.71,0.70,0.67,...,0.61,0.64,0.61,0.66,0.61,0.61,0.61,0.61,0.61,0.61
4,0.58,0.64,0.67,0.71,0.65,0.62,0.69,0.64,0.74,0.68,...,0.63,0.62,0.63,0.68,0.63,0.63,0.63,0.63,0.63,0.63
5,0.58,0.64,0.69,0.78,0.65,0.64,0.74,0.64,0.82,0.67,...,0.68,0.71,0.68,0.73,0.68,0.68,0.68,0.68,0.68,0.68


F1values

# Preparation for the table of ITSC paper

### preparation for RMSE table in overleaf


In [11]:
print("%.2f" %RMSE_original_mean['cam_d_yolo_v5'],'(' ,"%.2f" %RMSE_original_std['cam_d_yolo_v5'], ')&',"%.2f" %RMSE_fog_1_mean['cam_d_yolo_v5'],'(' ,"%.2f" %RMSE_fog_1_std['cam_d_yolo_v5'], ')&',"%.2f" %RMSE_fog_2_mean['cam_d_yolo_v5'],'(' ,"%.2f" %RMSE_fog_2_std['cam_d_yolo_v5'], ')&',"%.2f" %RMSE_fog_3_mean['cam_d_yolo_v5'],'(' ,"%.2f" %RMSE_fog_3_std['cam_d_yolo_v5'], ')&',"%.2f" %RMSE_rain_1_mean['cam_d_yolo_v5'],'(' ,"%.2f" %RMSE_rain_1_std['cam_d_yolo_v5'], ')&',"%.2f" %RMSE_rain_2_mean['cam_d_yolo_v5'],'(' ,"%.2f" %RMSE_rain_2_std['cam_d_yolo_v5'], ')&',"%.2f" %RMSE_rain_3_mean['cam_d_yolo_v5'],'(' ,"%.2f" %RMSE_rain_3_std['cam_d_yolo_v5'], ')\\',"\\")
print('&&Y8&', "%.2f" %RMSE_original_mean['cam_d_yolo_FT'],'(' ,"%.2f" %RMSE_original_std['cam_d_yolo_FT'], ')&',"%.2f" %RMSE_fog_1_mean['cam_d_yolo_FT'],'(' ,"%.2f" %RMSE_fog_1_std['cam_d_yolo_FT'], ')&',"%.2f" %RMSE_fog_2_mean['cam_d_yolo_FT'],'(' ,"%.2f" %RMSE_fog_2_std['cam_d_yolo_FT'], ')&',"%.2f" %RMSE_fog_3_mean['cam_d_yolo_FT'],'(' ,"%.2f" %RMSE_fog_3_std['cam_d_yolo_FT'], ')&',"%.2f" %RMSE_rain_1_mean['cam_d_yolo_FT'],'(' ,"%.2f" %RMSE_rain_1_std['cam_d_yolo_FT'], ')&',"%.2f" %RMSE_rain_2_mean['cam_d_yolo_FT'],'(' ,"%.2f" %RMSE_rain_2_std['cam_d_yolo_FT'], ')&',"%.2f" %RMSE_rain_3_mean['cam_d_yolo_FT'],'(' ,"%.2f" %RMSE_rain_3_std['cam_d_yolo_FT'], ')\\',"\\")
print('----------------------------------')
print("%.2f" %RMSE_original_mean['cam_w_yolo_v5'],'(' ,"%.2f" %RMSE_original_std['cam_w_yolo_v5'], ')&',"%.2f" %RMSE_fog_1_mean['cam_w_yolo_v5'],'(' ,"%.2f" %RMSE_fog_1_std['cam_w_yolo_v5'], ')&',"%.2f" %RMSE_fog_2_mean['cam_w_yolo_v5'],'(' ,"%.2f" %RMSE_fog_2_std['cam_w_yolo_v5'], ')&',"%.2f" %RMSE_fog_3_mean['cam_w_yolo_v5'],'(' ,"%.2f" %RMSE_fog_3_std['cam_w_yolo_v5'], ')&',"%.2f" %RMSE_rain_1_mean['cam_w_yolo_v5'],'(' ,"%.2f" %RMSE_rain_1_std['cam_w_yolo_v5'], ')&',"%.2f" %RMSE_rain_2_mean['cam_w_yolo_v5'],'(' ,"%.2f" %RMSE_rain_2_std['cam_w_yolo_v5'], ')&',"%.2f" %RMSE_rain_3_mean['cam_w_yolo_v5'],'(' ,"%.2f" %RMSE_rain_3_std['cam_w_yolo_v5'], ')\\',"\\")
print('&&Y8&',"%.2f" %RMSE_original_mean['cam_w_yolo_FT'],'(' ,"%.2f" %RMSE_original_std['cam_w_yolo_FT'], ')&',"%.2f" %RMSE_fog_1_mean['cam_w_yolo_FT'],'(' ,"%.2f" %RMSE_fog_1_std['cam_w_yolo_FT'], ')&',"%.2f" %RMSE_fog_2_mean['cam_w_yolo_FT'],'(' ,"%.2f" %RMSE_fog_2_std['cam_w_yolo_FT'], ')&',"%.2f" %RMSE_fog_3_mean['cam_w_yolo_FT'],'(' ,"%.2f" %RMSE_fog_3_std['cam_w_yolo_FT'], ')&',"%.2f" %RMSE_rain_1_mean['cam_w_yolo_FT'],'(' ,"%.2f" %RMSE_rain_1_std['cam_w_yolo_FT'], ')&',"%.2f" %RMSE_rain_2_mean['cam_w_yolo_FT'],'(' ,"%.2f" %RMSE_rain_2_std['cam_w_yolo_FT'], ')&',"%.2f" %RMSE_rain_3_mean['cam_w_yolo_FT'],'(' ,"%.2f" %RMSE_rain_3_std['cam_w_yolo_FT'], ')\\',"\\")
print('----------------------------------')
print("%.2f" %RMSE_original_mean['fusion'],'(' ,"%.2f" %RMSE_original_std['fusion'], ')&',"%.2f" %RMSE_fog_1_mean['fusion'],'(' ,"%.2f" %RMSE_fog_1_std['fusion'], ')&',"%.2f" %RMSE_fog_2_mean['fusion'],'(' ,"%.2f" %RMSE_fog_2_std['fusion'], ')&',"%.2f" %RMSE_fog_3_mean['fusion'],'(' ,"%.2f" %RMSE_fog_3_std['fusion'], ')&',"%.2f" %RMSE_rain_1_mean['fusion'],'(' ,"%.2f" %RMSE_rain_1_std['fusion'], ')&',"%.2f" %RMSE_rain_2_mean['fusion'],'(' ,"%.2f" %RMSE_rain_2_std['fusion'], ')&',"%.2f" %RMSE_rain_3_mean['fusion'],'(' ,"%.2f" %RMSE_rain_3_std['fusion'], ')\\',"\\")

1.26 ( 0.32 )& 1.25 ( 0.29 )& 1.16 ( 0.30 )& 1.13 ( 0.34 )& 1.28 ( 0.35 )& 1.13 ( 0.34 )& 1.13 ( 0.34 )\ \
&&Y8& 1.04 ( 0.30 )& 1.00 ( 0.20 )& 1.26 ( 0.24 )& 1.13 ( 0.34 )& 1.01 ( 0.28 )& 1.10 ( 0.33 )& 1.13 ( 0.34 )\ \
----------------------------------
1.08 ( 0.22 )& 1.20 ( 0.19 )& 1.23 ( 0.22 )& 1.13 ( 0.34 )& 1.11 ( 0.31 )& 1.13 ( 0.34 )& 1.13 ( 0.34 )\ \
&&Y8& 1.00 ( 0.30 )& 1.06 ( 0.32 )& 1.12 ( 0.33 )& 1.13 ( 0.34 )& 1.04 ( 0.31 )& 1.06 ( 0.30 )& 1.13 ( 0.34 )\ \
----------------------------------
0.88 ( 0.27 )& 0.87 ( 0.29 )& 1.06 ( 0.34 )& 1.13 ( 0.34 )& 0.93 ( 0.29 )& 1.00 ( 0.31 )& 1.03 ( 0.30 )\ \


In [12]:
print("%.2f" %RMSE_original_mean['cam_d_yolo_v5'],'(' ,"%.2f" %RMSE_original_std['cam_d_yolo_v5'], ')&',"%.2f" %RMSE_fog_1_mean['cam_d_yolo_v5'],'(' ,"%.2f" %RMSE_fog_1_std['cam_d_yolo_v5'], ')&',"%.2f" %RMSE_fog_2_mean['cam_d_yolo_v5'],'(' ,"%.2f" %RMSE_fog_2_std['cam_d_yolo_v5'], ')&', ')&',"%.2f" %RMSE_rain_1_mean['cam_d_yolo_v5'],'(' ,"%.2f" %RMSE_rain_1_std['cam_d_yolo_v5'], ')&',"%.2f" %RMSE_rain_2_mean['cam_d_yolo_v5'],'(' ,"%.2f" %RMSE_rain_2_std['cam_d_yolo_v5'], ')&',"%.2f" %RMSE_bright_2_mean['cam_d_yolo_v5'],'(' ,"%.2f" %RMSE_bright_2_std['cam_d_yolo_v5'], ')\\',"\\")
print('&&Y8&', "%.2f" %RMSE_original_mean['cam_d_yolo_FT'],'(' ,"%.2f" %RMSE_original_std['cam_d_yolo_FT'], ')&',"%.2f" %RMSE_fog_1_mean['cam_d_yolo_FT'],'(' ,"%.2f" %RMSE_fog_1_std['cam_d_yolo_FT'], ')&',"%.2f" %RMSE_fog_2_mean['cam_d_yolo_FT'],'(' ,"%.2f" %RMSE_fog_2_std['cam_d_yolo_FT'], ')&', ')&',"%.2f" %RMSE_rain_1_mean['cam_d_yolo_FT'],'(' ,"%.2f" %RMSE_rain_1_std['cam_d_yolo_FT'], ')&',"%.2f" %RMSE_rain_2_mean['cam_d_yolo_FT'],'(' ,"%.2f" %RMSE_rain_2_std['cam_d_yolo_FT'], ')&',"%.2f" %RMSE_bright_2_mean['cam_d_yolo_v5'],'(' ,"%.2f" %RMSE_bright_2_std['cam_d_yolo_v5'], ')\\',"\\")
print('----------------------------------')
print("%.2f" %RMSE_original_mean['cam_w_yolo_v5'],'(' ,"%.2f" %RMSE_original_std['cam_w_yolo_v5'], ')&',"%.2f" %RMSE_fog_1_mean['cam_w_yolo_v5'],'(' ,"%.2f" %RMSE_fog_1_std['cam_w_yolo_v5'], ')&',"%.2f" %RMSE_fog_2_mean['cam_w_yolo_v5'],'(' ,"%.2f" %RMSE_fog_2_std['cam_w_yolo_v5'], ')&', ')&',"%.2f" %RMSE_rain_1_mean['cam_w_yolo_v5'],'(' ,"%.2f" %RMSE_rain_1_std['cam_w_yolo_v5'], ')&',"%.2f" %RMSE_rain_2_mean['cam_w_yolo_v5'],'(' ,"%.2f" %RMSE_rain_2_std['cam_w_yolo_v5'], ')&',"%.2f" %RMSE_bright_2_mean['cam_d_yolo_v5'],'(' ,"%.2f" %RMSE_bright_2_std['cam_d_yolo_v5'], ')\\',"\\")
print('&&Y8&',"%.2f" %RMSE_original_mean['cam_w_yolo_FT'],'(' ,"%.2f" %RMSE_original_std['cam_w_yolo_FT'], ')&',"%.2f" %RMSE_fog_1_mean['cam_w_yolo_FT'],'(' ,"%.2f" %RMSE_fog_1_std['cam_w_yolo_FT'], ')&',"%.2f" %RMSE_fog_2_mean['cam_w_yolo_FT'],'(' ,"%.2f" %RMSE_fog_2_std['cam_w_yolo_FT'], ')&'"%.2f" %RMSE_rain_1_mean['cam_w_yolo_FT'],'(' ,"%.2f" %RMSE_rain_1_std['cam_w_yolo_FT'], ')&',"%.2f" %RMSE_rain_2_mean['cam_w_yolo_FT'],'(' ,"%.2f" %RMSE_rain_2_std['cam_w_yolo_FT'], ')&',"%.2f" %RMSE_bright_2_mean['cam_d_yolo_v5'],'(' ,"%.2f" %RMSE_bright_2_std['cam_d_yolo_v5'],')\\',"\\")
print('----------------------------------')
print("%.2f" %RMSE_original_mean['fusion'],'(' ,"%.2f" %RMSE_original_std['fusion'], ')&',"%.2f" %RMSE_fog_1_mean['fusion'],'(' ,"%.2f" %RMSE_fog_1_std['fusion'], ')&',"%.2f" %RMSE_fog_2_mean['fusion'],'(' ,"%.2f" %RMSE_fog_2_std['fusion'], ')&', ')&',"%.2f" %RMSE_rain_1_mean['fusion'],'(' ,"%.2f" %RMSE_rain_1_std['fusion'], ')&',"%.2f" %RMSE_rain_2_mean['fusion'],'(' ,"%.2f" %RMSE_rain_2_std['fusion'], ')&',"%.2f" %RMSE_bright_2_mean['cam_d_yolo_v5'],'(' ,"%.2f" %RMSE_bright_2_std['cam_d_yolo_v5'], ')\\',"\\")


1.26 ( 0.32 )& 1.25 ( 0.29 )& 1.16 ( 0.30 )& )& 1.28 ( 0.35 )& 1.13 ( 0.34 )& 1.27 ( 0.35 )\ \
&&Y8& 1.04 ( 0.30 )& 1.00 ( 0.20 )& 1.26 ( 0.24 )& )& 1.01 ( 0.28 )& 1.10 ( 0.33 )& 1.27 ( 0.35 )\ \
----------------------------------
1.08 ( 0.22 )& 1.20 ( 0.19 )& 1.23 ( 0.22 )& )& 1.11 ( 0.31 )& 1.13 ( 0.34 )& 1.27 ( 0.35 )\ \
&&Y8& 1.00 ( 0.30 )& 1.06 ( 0.32 )& 1.12 ( 0.33 )&1.04 ( 0.31 )& 1.06 ( 0.30 )& 1.27 ( 0.35 )\ \
----------------------------------
0.88 ( 0.27 )& 0.87 ( 0.29 )& 1.06 ( 0.34 )& )& 0.93 ( 0.29 )& 1.00 ( 0.31 )& 1.27 ( 0.35 )\ \


# RMSE Dark e brigth

In [13]:
print("%.2f" %RMSE_original_mean['cam_d_yolo_v5'],'(' ,"%.2f" %RMSE_original_std['cam_d_yolo_v5'], ')&',"%.2f" %RMSE_bright_1_mean['cam_d_yolo_v5'],'(' ,"%.2f" %RMSE_bright_1_std['cam_d_yolo_v5'], ')&',"%.2f" %RMSE_bright_2_mean['cam_d_yolo_v5'],'(' ,"%.2f" %RMSE_bright_2_std['cam_d_yolo_v5'], ')&',"%.2f" %RMSE_bright_3_mean['cam_d_yolo_v5'],'(' ,"%.2f" %RMSE_bright_3_std['cam_d_yolo_v5'], ')&',"%.2f" %RMSE_dark_1_mean['cam_d_yolo_v5'],'(' ,"%.2f" %RMSE_dark_1_std['cam_d_yolo_v5'], ')&',"%.2f" %RMSE_dark_2_mean['cam_d_yolo_v5'],'(' ,"%.2f" %RMSE_dark_2_std['cam_d_yolo_v5'], ')&',"%.2f" %RMSE_dark_3_mean['cam_d_yolo_v5'],'(' ,"%.2f" %RMSE_dark_3_std['cam_d_yolo_v5'], ')\\',"\\")
print('&&Y8&', "%.2f" %RMSE_original_mean['cam_d_yolo_FT'],'(' ,"%.2f" %RMSE_original_std['cam_d_yolo_FT'], ')&',"%.2f" %RMSE_bright_1_mean['cam_d_yolo_FT'],'(' ,"%.2f" %RMSE_bright_1_std['cam_d_yolo_FT'], ')&',"%.2f" %RMSE_bright_2_mean['cam_d_yolo_FT'],'(' ,"%.2f" %RMSE_bright_2_std['cam_d_yolo_FT'], ')&',"%.2f" %RMSE_bright_3_mean['cam_d_yolo_FT'],'(' ,"%.2f" %RMSE_bright_3_std['cam_d_yolo_FT'], ')&',"%.2f" %RMSE_dark_1_mean['cam_d_yolo_FT'],'(' ,"%.2f" %RMSE_dark_1_std['cam_d_yolo_FT'], ')&',"%.2f" %RMSE_dark_2_mean['cam_d_yolo_FT'],'(' ,"%.2f" %RMSE_dark_2_std['cam_d_yolo_FT'], ')&',"%.2f" %RMSE_dark_3_mean['cam_d_yolo_FT'],'(' ,"%.2f" %RMSE_dark_3_std['cam_d_yolo_FT'], ')\\',"\\")
print('----------------------------------')
print("%.2f" %RMSE_original_mean['cam_w_yolo_v5'],'(' ,"%.2f" %RMSE_original_std['cam_w_yolo_v5'], ')&',"%.2f" %RMSE_bright_1_mean['cam_w_yolo_v5'],'(' ,"%.2f" %RMSE_bright_1_std['cam_w_yolo_v5'], ')&',"%.2f" %RMSE_bright_2_mean['cam_w_yolo_v5'],'(' ,"%.2f" %RMSE_bright_2_std['cam_w_yolo_v5'], ')&',"%.2f" %RMSE_bright_3_mean['cam_w_yolo_v5'],'(' ,"%.2f" %RMSE_bright_3_std['cam_w_yolo_v5'], ')&',"%.2f" %RMSE_dark_1_mean['cam_w_yolo_v5'],'(' ,"%.2f" %RMSE_dark_1_std['cam_w_yolo_v5'], ')&',"%.2f" %RMSE_dark_2_mean['cam_w_yolo_v5'],'(' ,"%.2f" %RMSE_dark_2_std['cam_w_yolo_v5'], ')&',"%.2f" %RMSE_dark_3_mean['cam_w_yolo_v5'],'(' ,"%.2f" %RMSE_dark_3_std['cam_w_yolo_v5'], ')\\',"\\")
print('&&Y8&',"%.2f" %RMSE_original_mean['cam_w_yolo_FT'],'(' ,"%.2f" %RMSE_original_std['cam_w_yolo_FT'], ')&',"%.2f" %RMSE_bright_1_mean['cam_w_yolo_FT'],'(' ,"%.2f" %RMSE_bright_1_std['cam_w_yolo_FT'], ')&',"%.2f" %RMSE_bright_2_mean['cam_w_yolo_FT'],'(' ,"%.2f" %RMSE_bright_2_std['cam_w_yolo_FT'], ')&',"%.2f" %RMSE_bright_3_mean['cam_w_yolo_FT'],'(' ,"%.2f" %RMSE_bright_3_std['cam_w_yolo_FT'], ')&',"%.2f" %RMSE_dark_1_mean['cam_w_yolo_FT'],'(' ,"%.2f" %RMSE_dark_1_std['cam_w_yolo_FT'], ')&',"%.2f" %RMSE_dark_2_mean['cam_w_yolo_FT'],'(' ,"%.2f" %RMSE_dark_2_std['cam_w_yolo_FT'], ')&',"%.2f" %RMSE_dark_3_mean['cam_w_yolo_FT'],'(' ,"%.2f" %RMSE_dark_3_std['cam_w_yolo_FT'], ')\\',"\\")
print('----------------------------------')
print("%.2f" %RMSE_original_mean['fusion'],'(' ,"%.2f" %RMSE_original_std['fusion'], ')&',"%.2f" %RMSE_bright_1_mean['fusion'],'(' ,"%.2f" %RMSE_bright_1_std['fusion'], ')&',"%.2f" %RMSE_bright_2_mean['fusion'],'(' ,"%.2f" %RMSE_bright_2_std['fusion'], ')&',"%.2f" %RMSE_bright_3_mean['fusion'],'(' ,"%.2f" %RMSE_bright_3_std['fusion'], ')&',"%.2f" %RMSE_dark_1_mean['fusion'],'(' ,"%.2f" %RMSE_dark_1_std['fusion'], ')&',"%.2f" %RMSE_dark_2_mean['fusion'],'(' ,"%.2f" %RMSE_dark_2_std['fusion'], ')&',"%.2f" %RMSE_dark_3_mean['fusion'],'(' ,"%.2f" %RMSE_dark_3_std['fusion'], ')\\',"\\")

1.26 ( 0.32 )& 1.22 ( 0.30 )& 1.27 ( 0.35 )& 1.26 ( 0.38 )& 1.26 ( 0.26 )& 1.28 ( 0.27 )& 1.22 ( 0.24 )\ \
&&Y8& 1.04 ( 0.30 )& 0.95 ( 0.31 )& 0.86 ( 0.20 )& 0.83 ( 0.17 )& 1.09 ( 0.30 )& 1.10 ( 0.30 )& 1.13 ( 0.32 )\ \
----------------------------------
1.08 ( 0.22 )& 1.13 ( 0.21 )& 1.17 ( 0.25 )& 1.17 ( 0.25 )& 1.08 ( 0.26 )& 1.07 ( 0.24 )& 1.05 ( 0.26 )\ \
&&Y8& 1.00 ( 0.30 )& 0.97 ( 0.29 )& 0.91 ( 0.29 )& 0.89 ( 0.28 )& 1.04 ( 0.31 )& 1.08 ( 0.31 )& 1.09 ( 0.33 )\ \
----------------------------------
0.88 ( 0.27 )& 0.86 ( 0.29 )& 0.81 ( 0.29 )& 0.81 ( 0.30 )& 0.93 ( 0.27 )& 0.97 ( 0.29 )& 0.96 ( 0.29 )\ \


### preparation for F1_score table in overleaf


In [14]:
print("%.2f" %F1_score_original_mean['cam_d_yolo_v5'],'(' ,"%.2f" %F1_score_original_std['cam_d_yolo_v5'], ')&',"%.2f" %F1_score_fog_1_mean['cam_d_yolo_v5'],'(' ,"%.2f" %F1_score_fog_1_std['cam_d_yolo_v5'], ')&',"%.2f" %F1_score_fog_2_mean['cam_d_yolo_v5'],'(' ,"%.2f" %F1_score_fog_2_std['cam_d_yolo_v5'], ')&',"%.2f" %F1_score_fog_3_mean['cam_d_yolo_v5'],'(' ,"%.2f" %F1_score_fog_3_std['cam_d_yolo_v5'], ')&',"%.2f" %F1_score_rain_1_mean['cam_d_yolo_v5'],'(' ,"%.2f" %F1_score_rain_1_std['cam_d_yolo_v5'], ')&',"%.2f" %F1_score_rain_2_mean['cam_d_yolo_v5'],'(' ,"%.2f" %F1_score_rain_2_std['cam_d_yolo_v5'], ')&',"%.2f" %F1_score_rain_3_mean['cam_d_yolo_v5'],'(' ,"%.2f" %F1_score_rain_3_std['cam_d_yolo_v5'], ')\\',"\\")
print('&&Y8&', "%.2f" %F1_score_original_mean['cam_d_yolo_FT'],'(' ,"%.2f" %F1_score_original_std['cam_d_yolo_FT'], ')&',"%.2f" %F1_score_fog_1_mean['cam_d_yolo_FT'],'(' ,"%.2f" %F1_score_fog_1_std['cam_d_yolo_FT'], ')&',"%.2f" %F1_score_fog_2_mean['cam_d_yolo_FT'],'(' ,"%.2f" %F1_score_fog_2_std['cam_d_yolo_FT'], ')&',"%.2f" %F1_score_fog_3_mean['cam_d_yolo_FT'],'(' ,"%.2f" %F1_score_fog_3_std['cam_d_yolo_FT'], ')&',"%.2f" %F1_score_rain_1_mean['cam_d_yolo_FT'],'(' ,"%.2f" %F1_score_rain_1_std['cam_d_yolo_FT'], ')&',"%.2f" %F1_score_rain_2_mean['cam_d_yolo_FT'],'(' ,"%.2f" %F1_score_rain_2_std['cam_d_yolo_FT'], ')&',"%.2f" %F1_score_rain_3_mean['cam_d_yolo_FT'],'(' ,"%.2f" %F1_score_rain_3_std['cam_d_yolo_FT'], ')\\',"\\")
print('----------------------------------')
print("%.2f" %F1_score_original_mean['cam_w_yolo_v5'],'(' ,"%.2f" %F1_score_original_std['cam_w_yolo_v5'], ')&',"%.2f" %F1_score_fog_1_mean['cam_w_yolo_v5'],'(' ,"%.2f" %F1_score_fog_1_std['cam_w_yolo_v5'], ')&',"%.2f" %F1_score_fog_2_mean['cam_w_yolo_v5'],'(' ,"%.2f" %F1_score_fog_2_std['cam_w_yolo_v5'], ')&',"%.2f" %F1_score_fog_3_mean['cam_w_yolo_v5'],'(' ,"%.2f" %F1_score_fog_3_std['cam_w_yolo_v5'], ')&',"%.2f" %F1_score_rain_1_mean['cam_w_yolo_v5'],'(' ,"%.2f" %F1_score_rain_1_std['cam_w_yolo_v5'], ')&',"%.2f" %F1_score_rain_2_mean['cam_w_yolo_v5'],'(' ,"%.2f" %F1_score_rain_2_std['cam_w_yolo_v5'], ')&',"%.2f" %F1_score_rain_3_mean['cam_w_yolo_v5'],'(' ,"%.2f" %F1_score_rain_3_std['cam_w_yolo_v5'], ')\\',"\\")
print('&&Y8&',"%.2f" %F1_score_original_mean['cam_w_yolo_FT'],'(' ,"%.2f" %F1_score_original_std['cam_w_yolo_FT'], ')&',"%.2f" %F1_score_fog_1_mean['cam_w_yolo_FT'],'(' ,"%.2f" %F1_score_fog_1_std['cam_w_yolo_FT'], ')&',"%.2f" %F1_score_fog_2_mean['cam_w_yolo_FT'],'(' ,"%.2f" %F1_score_fog_2_std['cam_w_yolo_FT'], ')&',"%.2f" %F1_score_fog_3_mean['cam_w_yolo_FT'],'(' ,"%.2f" %F1_score_fog_3_std['cam_w_yolo_FT'], ')&',"%.2f" %F1_score_rain_1_mean['cam_w_yolo_FT'],'(' ,"%.2f" %F1_score_rain_1_std['cam_w_yolo_FT'], ')&',"%.2f" %F1_score_rain_2_mean['cam_w_yolo_FT'],'(' ,"%.2f" %F1_score_rain_2_std['cam_w_yolo_FT'], ')&',"%.2f" %F1_score_rain_3_mean['cam_w_yolo_FT'],'(' ,"%.2f" %F1_score_rain_3_std['cam_w_yolo_FT'], ')\\',"\\")
print('----------------------------------')
print("%.2f" %F1_score_original_mean['fusion'],'(' ,"%.2f" %F1_score_original_std['fusion'], ')&',"%.2f" %F1_score_fog_1_mean['fusion'],'(' ,"%.2f" %F1_score_fog_1_std['fusion'], ')&',"%.2f" %F1_score_fog_2_mean['fusion'],'(' ,"%.2f" %F1_score_fog_2_std['fusion'], ')&',"%.2f" %F1_score_fog_3_mean['fusion'],'(' ,"%.2f" %F1_score_fog_3_std['fusion'], ')&',"%.2f" %F1_score_rain_1_mean['fusion'],'(' ,"%.2f" %F1_score_rain_1_std['fusion'], ')&',"%.2f" %F1_score_rain_2_mean['fusion'],'(' ,"%.2f" %F1_score_rain_2_std['fusion'], ')&',"%.2f" %F1_score_rain_3_mean['fusion'],'(' ,"%.2f" %F1_score_rain_3_std['fusion'], ')\\',"\\")

41.58 ( 14.88 )& 41.25 ( 8.99 )& 9.44 ( 8.42 )& 0.00 ( 0.00 )& 39.14 ( 12.18 )& 0.00 ( 0.00 )& 0.00 ( 0.00 )\ \
&&Y8& 17.50 ( 3.73 )& 31.04 ( 7.19 )& 14.25 ( 7.80 )& 0.00 ( 0.00 )& 33.03 ( 9.62 )& 9.01 ( 6.55 )& 0.00 ( 0.00 )\ \
----------------------------------
35.71 ( 11.40 )& 30.58 ( 7.77 )& 12.75 ( 5.25 )& 0.00 ( 0.00 )& 13.76 ( 4.98 )& 0.00 ( 0.00 )& 0.00 ( 0.00 )\ \
&&Y8& 36.40 ( 16.52 )& 19.76 ( 11.60 )& 1.73 ( 1.97 )& 0.00 ( 0.00 )& 25.89 ( 13.34 )& 20.33 ( 10.32 )& 0.51 ( 1.24 )\ \
----------------------------------
18.45 ( 3.28 )& 26.61 ( 1.84 )& 2.07 ( 2.64 )& 0.00 ( 0.00 )& 9.76 ( 4.46 )& 0.00 ( 0.00 )& 0.00 ( 0.00 )\ \


# F1 Dark e brigth

In [15]:
print("%.2f" %F1_score_original_mean['cam_d_yolo_v5'],'(' ,"%.2f" %F1_score_original_std['cam_d_yolo_v5'], ')&',"%.2f" %F1_score_bright_1_mean['cam_d_yolo_v5'],'(' ,"%.2f" %F1_score_bright_1_std['cam_d_yolo_v5'], ')&',"%.2f" %F1_score_bright_2_mean['cam_d_yolo_v5'],'(' ,"%.2f" %F1_score_bright_2_std['cam_d_yolo_v5'], ')&',"%.2f" %F1_score_bright_3_mean['cam_d_yolo_v5'],'(' ,"%.2f" %F1_score_bright_3_std['cam_d_yolo_v5'], ')&',"%.2f" %F1_score_dark_1_mean['cam_d_yolo_v5'],'(' ,"%.2f" %F1_score_dark_1_std['cam_d_yolo_v5'], ')&',"%.2f" %F1_score_dark_2_mean['cam_d_yolo_v5'],'(' ,"%.2f" %F1_score_dark_2_std['cam_d_yolo_v5'], ')&',"%.2f" %F1_score_dark_3_mean['cam_d_yolo_v5'],'(' ,"%.2f" %F1_score_dark_3_std['cam_d_yolo_v5'], ')\\',"\\")
print('&&Y8&', "%.2f" %F1_score_original_mean['cam_d_yolo_FT'],'(' ,"%.2f" %F1_score_original_std['cam_d_yolo_FT'], ')&',"%.2f" %F1_score_bright_1_mean['cam_d_yolo_FT'],'(' ,"%.2f" %F1_score_bright_1_std['cam_d_yolo_FT'], ')&',"%.2f" %F1_score_bright_2_mean['cam_d_yolo_FT'],'(' ,"%.2f" %F1_score_bright_2_std['cam_d_yolo_FT'], ')&',"%.2f" %F1_score_bright_3_mean['cam_d_yolo_FT'],'(' ,"%.2f" %F1_score_bright_3_std['cam_d_yolo_FT'], ')&',"%.2f" %F1_score_dark_1_mean['cam_d_yolo_FT'],'(' ,"%.2f" %F1_score_dark_1_std['cam_d_yolo_FT'], ')&',"%.2f" %F1_score_dark_2_mean['cam_d_yolo_FT'],'(' ,"%.2f" %F1_score_dark_2_std['cam_d_yolo_FT'], ')&',"%.2f" %F1_score_dark_3_mean['cam_d_yolo_FT'],'(' ,"%.2f" %F1_score_dark_3_std['cam_d_yolo_FT'], ')\\',"\\")
print('----------------------------------')
print("%.2f" %F1_score_original_mean['cam_w_yolo_v5'],'(' ,"%.2f" %F1_score_original_std['cam_w_yolo_v5'], ')&',"%.2f" %F1_score_bright_1_mean['cam_w_yolo_v5'],'(' ,"%.2f" %F1_score_bright_1_std['cam_w_yolo_v5'], ')&',"%.2f" %F1_score_bright_2_mean['cam_w_yolo_v5'],'(' ,"%.2f" %F1_score_bright_2_std['cam_w_yolo_v5'], ')&',"%.2f" %F1_score_bright_3_mean['cam_w_yolo_v5'],'(' ,"%.2f" %F1_score_bright_3_std['cam_w_yolo_v5'], ')&',"%.2f" %F1_score_dark_1_mean['cam_w_yolo_v5'],'(' ,"%.2f" %F1_score_dark_1_std['cam_w_yolo_v5'], ')&',"%.2f" %F1_score_dark_2_mean['cam_w_yolo_v5'],'(' ,"%.2f" %F1_score_dark_2_std['cam_w_yolo_v5'], ')&',"%.2f" %F1_score_dark_3_mean['cam_w_yolo_v5'],'(' ,"%.2f" %F1_score_dark_3_std['cam_w_yolo_v5'], ')\\',"\\")
print('&&Y8&',"%.2f" %F1_score_original_mean['cam_w_yolo_FT'],'(' ,"%.2f" %F1_score_original_std['cam_w_yolo_FT'], ')&',"%.2f" %F1_score_bright_1_mean['cam_w_yolo_FT'],'(' ,"%.2f" %F1_score_bright_1_std['cam_w_yolo_FT'], ')&',"%.2f" %F1_score_bright_2_mean['cam_w_yolo_FT'],'(' ,"%.2f" %F1_score_bright_2_std['cam_w_yolo_FT'], ')&',"%.2f" %F1_score_bright_3_mean['cam_w_yolo_FT'],'(' ,"%.2f" %F1_score_bright_3_std['cam_w_yolo_FT'], ')&',"%.2f" %F1_score_dark_1_mean['cam_w_yolo_FT'],'(' ,"%.2f" %F1_score_dark_1_std['cam_w_yolo_FT'], ')&',"%.2f" %F1_score_dark_2_mean['cam_w_yolo_FT'],'(' ,"%.2f" %F1_score_dark_2_std['cam_w_yolo_FT'], ')&',"%.2f" %F1_score_dark_3_mean['cam_w_yolo_FT'],'(' ,"%.2f" %F1_score_dark_3_std['cam_w_yolo_FT'], ')\\',"\\")
print('----------------------------------')
print("%.2f" %F1_score_original_mean['fusion'],'(' ,"%.2f" %F1_score_original_std['fusion'], ')&',"%.2f" %F1_score_bright_1_mean['fusion'],'(' ,"%.2f" %F1_score_bright_1_std['fusion'], ')&',"%.2f" %F1_score_bright_2_mean['fusion'],'(' ,"%.2f" %F1_score_bright_2_std['fusion'], ')&',"%.2f" %F1_score_bright_3_mean['fusion'],'(' ,"%.2f" %F1_score_bright_3_std['fusion'], ')&',"%.2f" %F1_score_dark_1_mean['fusion'],'(' ,"%.2f" %F1_score_dark_1_std['fusion'], ')&',"%.2f" %F1_score_dark_2_mean['fusion'],'(' ,"%.2f" %F1_score_dark_2_std['fusion'], ')&',"%.2f" %F1_score_dark_3_mean['fusion'],'(' ,"%.2f" %F1_score_dark_3_std['fusion'], ')\\',"\\")

41.58 ( 14.88 )& 47.75 ( 16.16 )& 49.45 ( 14.90 )& 49.63 ( 16.33 )& 40.12 ( 12.42 )& 36.62 ( 11.28 )& 45.75 ( 12.24 )\ \
&&Y8& 17.50 ( 3.73 )& 34.77 ( 13.92 )& 54.82 ( 18.09 )& 57.55 ( 18.96 )& 13.52 ( 8.42 )& 14.42 ( 11.19 )& 4.35 ( 8.49 )\ \
----------------------------------
35.71 ( 11.40 )& 28.08 ( 9.08 )& 19.87 ( 6.98 )& 22.71 ( 8.11 )& 34.61 ( 9.31 )& 34.26 ( 12.45 )& 32.28 ( 8.69 )\ \
&&Y8& 36.40 ( 16.52 )& 42.20 ( 20.99 )& 44.49 ( 21.44 )& 46.98 ( 22.29 )& 25.82 ( 13.06 )& 15.36 ( 7.57 )& 11.88 ( 6.81 )\ \
----------------------------------
18.45 ( 3.28 )& 22.36 ( 8.03 )& 29.17 ( 9.75 )& 27.63 ( 11.61 )& 17.23 ( 3.53 )& 15.26 ( 4.24 )& 10.19 ( 4.19 )\ \


### preparation for Accuracy_score table in overleaf


In [16]:
print("%.2f" %Accuracy_score_original_mean['cam_d_yolo_v5'],'(' ,"%.2f" %Accuracy_score_original_std['cam_d_yolo_v5'], ')&',"%.2f" %Accuracy_score_fog_1_mean['cam_d_yolo_v5'],'(' ,"%.2f" %Accuracy_score_fog_1_std['cam_d_yolo_v5'], ')&',"%.2f" %Accuracy_score_fog_2_mean['cam_d_yolo_v5'],'(' ,"%.2f" %Accuracy_score_fog_2_std['cam_d_yolo_v5'], ')&',"%.2f" %Accuracy_score_fog_3_mean['cam_d_yolo_v5'],'(' ,"%.2f" %Accuracy_score_fog_3_std['cam_d_yolo_v5'], ')&',"%.2f" %Accuracy_score_rain_1_mean['cam_d_yolo_v5'],'(' ,"%.2f" %Accuracy_score_rain_1_std['cam_d_yolo_v5'], ')&',"%.2f" %Accuracy_score_rain_2_mean['cam_d_yolo_v5'],'(' ,"%.2f" %Accuracy_score_rain_2_std['cam_d_yolo_v5'], ')&',"%.2f" %Accuracy_score_rain_3_mean['cam_d_yolo_v5'],'(' ,"%.2f" %Accuracy_score_rain_3_std['cam_d_yolo_v5'], ')\\',"\\")
print('&&Y8&', "%.2f" %Accuracy_score_original_mean['cam_d_yolo_FT'],'(' ,"%.2f" %Accuracy_score_original_std['cam_d_yolo_FT'], ')&',"%.2f" %Accuracy_score_fog_1_mean['cam_d_yolo_FT'],'(' ,"%.2f" %Accuracy_score_fog_1_std['cam_d_yolo_FT'], ')&',"%.2f" %Accuracy_score_fog_2_mean['cam_d_yolo_FT'],'(' ,"%.2f" %Accuracy_score_fog_2_std['cam_d_yolo_FT'], ')&',"%.2f" %Accuracy_score_fog_3_mean['cam_d_yolo_FT'],'(' ,"%.2f" %Accuracy_score_fog_3_std['cam_d_yolo_FT'], ')&',"%.2f" %Accuracy_score_rain_1_mean['cam_d_yolo_FT'],'(' ,"%.2f" %Accuracy_score_rain_1_std['cam_d_yolo_FT'], ')&',"%.2f" %Accuracy_score_rain_2_mean['cam_d_yolo_FT'],'(' ,"%.2f" %Accuracy_score_rain_2_std['cam_d_yolo_FT'], ')&',"%.2f" %Accuracy_score_rain_3_mean['cam_d_yolo_FT'],'(' ,"%.2f" %Accuracy_score_rain_3_std['cam_d_yolo_FT'], ')\\',"\\")
print('----------------------------------')
print("%.2f" %Accuracy_score_original_mean['cam_w_yolo_v5'],'(' ,"%.2f" %Accuracy_score_original_std['cam_w_yolo_v5'], ')&',"%.2f" %Accuracy_score_fog_1_mean['cam_w_yolo_v5'],'(' ,"%.2f" %Accuracy_score_fog_1_std['cam_w_yolo_v5'], ')&',"%.2f" %Accuracy_score_fog_2_mean['cam_w_yolo_v5'],'(' ,"%.2f" %Accuracy_score_fog_2_std['cam_w_yolo_v5'], ')&',"%.2f" %Accuracy_score_fog_3_mean['cam_w_yolo_v5'],'(' ,"%.2f" %Accuracy_score_fog_3_std['cam_w_yolo_v5'], ')&',"%.2f" %Accuracy_score_rain_1_mean['cam_w_yolo_v5'],'(' ,"%.2f" %Accuracy_score_rain_1_std['cam_w_yolo_v5'], ')&',"%.2f" %Accuracy_score_rain_2_mean['cam_w_yolo_v5'],'(' ,"%.2f" %Accuracy_score_rain_2_std['cam_w_yolo_v5'], ')&',"%.2f" %Accuracy_score_rain_3_mean['cam_w_yolo_v5'],'(' ,"%.2f" %Accuracy_score_rain_3_std['cam_w_yolo_v5'], ')\\',"\\")
print('&&Y8&',"%.2f" %Accuracy_score_original_mean['cam_w_yolo_FT'],'(' ,"%.2f" %Accuracy_score_original_std['cam_w_yolo_FT'], ')&',"%.2f" %Accuracy_score_fog_1_mean['cam_w_yolo_FT'],'(' ,"%.2f" %Accuracy_score_fog_1_std['cam_w_yolo_FT'], ')&',"%.2f" %Accuracy_score_fog_2_mean['cam_w_yolo_FT'],'(' ,"%.2f" %Accuracy_score_fog_2_std['cam_w_yolo_FT'], ')&',"%.2f" %Accuracy_score_fog_3_mean['cam_w_yolo_FT'],'(' ,"%.2f" %Accuracy_score_fog_3_std['cam_w_yolo_FT'], ')&',"%.2f" %Accuracy_score_rain_1_mean['cam_w_yolo_FT'],'(' ,"%.2f" %Accuracy_score_rain_1_std['cam_w_yolo_FT'], ')&',"%.2f" %Accuracy_score_rain_2_mean['cam_w_yolo_FT'],'(' ,"%.2f" %Accuracy_score_rain_2_std['cam_w_yolo_FT'], ')&',"%.2f" %Accuracy_score_rain_3_mean['cam_w_yolo_FT'],'(' ,"%.2f" %Accuracy_score_rain_3_std['cam_w_yolo_FT'], ')\\',"\\")
print('----------------------------------')
print("%.2f" %Accuracy_score_original_mean['fusion'],'(' ,"%.2f" %Accuracy_score_original_std['fusion'], ')&',"%.2f" %Accuracy_score_fog_1_mean['fusion'],'(' ,"%.2f" %Accuracy_score_fog_1_std['fusion'], ')&',"%.2f" %Accuracy_score_fog_2_mean['fusion'],'(' ,"%.2f" %Accuracy_score_fog_2_std['fusion'], ')&',"%.2f" %Accuracy_score_fog_3_mean['fusion'],'(' ,"%.2f" %Accuracy_score_fog_3_std['fusion'], ')&',"%.2f" %Accuracy_score_rain_1_mean['fusion'],'(' ,"%.2f" %Accuracy_score_rain_1_std['fusion'], ')&',"%.2f" %Accuracy_score_rain_2_mean['fusion'],'(' ,"%.2f" %Accuracy_score_rain_2_std['fusion'], ')&',"%.2f" %Accuracy_score_rain_3_mean['fusion'],'(' ,"%.2f" %Accuracy_score_rain_3_std['fusion'], ')\\',"\\")

67.95 ( 11.02 )& 68.75 ( 9.60 )& 62.62 ( 18.12 )& 62.34 ( 18.89 )& 68.04 ( 9.43 )& 62.34 ( 18.89 )& 62.34 ( 18.89 )\ \
&&Y8& 63.51 ( 15.39 )& 66.43 ( 12.87 )& 62.26 ( 16.10 )& 62.34 ( 18.89 )& 67.40 ( 12.50 )& 63.61 ( 17.59 )& 62.34 ( 18.89 )\ \
----------------------------------
68.49 ( 11.52 )& 65.61 ( 14.55 )& 63.16 ( 15.88 )& 62.34 ( 18.89 )& 64.50 ( 16.78 )& 62.34 ( 18.89 )& 62.34 ( 18.89 )\ \
&&Y8& 70.89 ( 11.13 )& 66.35 ( 15.41 )& 62.73 ( 18.68 )& 62.34 ( 18.89 )& 67.68 ( 13.03 )& 66.94 ( 14.65 )& 62.07 ( 18.14 )\ \
----------------------------------
64.42 ( 15.08 )& 66.31 ( 14.49 )& 62.29 ( 18.51 )& 62.34 ( 18.89 )& 63.79 ( 16.84 )& 62.34 ( 18.89 )& 62.34 ( 18.89 )\ \


# Accuracy Dark e brigth


In [17]:
print("%.2f" %Accuracy_score_original_mean['cam_d_yolo_v5'],'(' ,"%.2f" %Accuracy_score_original_std['cam_d_yolo_v5'], ')&',"%.2f" %Accuracy_score_bright_1_mean['cam_d_yolo_v5'],'(' ,"%.2f" %Accuracy_score_bright_1_std['cam_d_yolo_v5'], ')&',"%.2f" %Accuracy_score_bright_2_mean['cam_d_yolo_v5'],'(' ,"%.2f" %Accuracy_score_bright_2_std['cam_d_yolo_v5'], ')&',"%.2f" %Accuracy_score_bright_3_mean['cam_d_yolo_v5'],'(' ,"%.2f" %Accuracy_score_bright_3_std['cam_d_yolo_v5'], ')&',"%.2f" %Accuracy_score_dark_1_mean['cam_d_yolo_v5'],'(' ,"%.2f" %Accuracy_score_dark_1_std['cam_d_yolo_v5'], ')&',"%.2f" %Accuracy_score_dark_2_mean['cam_d_yolo_v5'],'(' ,"%.2f" %Accuracy_score_dark_2_std['cam_d_yolo_v5'], ')&',"%.2f" %Accuracy_score_dark_3_mean['cam_d_yolo_v5'],'(' ,"%.2f" %Accuracy_score_dark_3_std['cam_d_yolo_v5'], ')\\',"\\")
print('&&Y8&', "%.2f" %Accuracy_score_original_mean['cam_d_yolo_FT'],'(' ,"%.2f" %Accuracy_score_original_std['cam_d_yolo_FT'], ')&',"%.2f" %Accuracy_score_bright_1_mean['cam_d_yolo_FT'],'(' ,"%.2f" %Accuracy_score_bright_1_std['cam_d_yolo_FT'], ')&',"%.2f" %Accuracy_score_bright_2_mean['cam_d_yolo_FT'],'(' ,"%.2f" %Accuracy_score_bright_2_std['cam_d_yolo_FT'], ')&',"%.2f" %Accuracy_score_bright_3_mean['cam_d_yolo_FT'],'(' ,"%.2f" %Accuracy_score_bright_3_std['cam_d_yolo_FT'], ')&',"%.2f" %Accuracy_score_dark_1_mean['cam_d_yolo_FT'],'(' ,"%.2f" %Accuracy_score_dark_1_std['cam_d_yolo_FT'], ')&',"%.2f" %Accuracy_score_dark_2_mean['cam_d_yolo_FT'],'(' ,"%.2f" %Accuracy_score_dark_2_std['cam_d_yolo_FT'], ')&',"%.2f" %Accuracy_score_dark_3_mean['cam_d_yolo_FT'],'(' ,"%.2f" %Accuracy_score_dark_3_std['cam_d_yolo_FT'], ')\\',"\\")
print('----------------------------------')
print("%.2f" %Accuracy_score_original_mean['cam_w_yolo_v5'],'(' ,"%.2f" %Accuracy_score_original_std['cam_w_yolo_v5'], ')&',"%.2f" %Accuracy_score_bright_1_mean['cam_w_yolo_v5'],'(' ,"%.2f" %Accuracy_score_bright_1_std['cam_w_yolo_v5'], ')&',"%.2f" %Accuracy_score_bright_2_mean['cam_w_yolo_v5'],'(' ,"%.2f" %Accuracy_score_bright_2_std['cam_w_yolo_v5'], ')&',"%.2f" %Accuracy_score_bright_3_mean['cam_w_yolo_v5'],'(' ,"%.2f" %Accuracy_score_bright_3_std['cam_w_yolo_v5'], ')&',"%.2f" %Accuracy_score_dark_1_mean['cam_w_yolo_v5'],'(' ,"%.2f" %Accuracy_score_dark_1_std['cam_w_yolo_v5'], ')&',"%.2f" %Accuracy_score_dark_2_mean['cam_w_yolo_v5'],'(' ,"%.2f" %Accuracy_score_dark_2_std['cam_w_yolo_v5'], ')&',"%.2f" %Accuracy_score_dark_3_mean['cam_w_yolo_v5'],'(' ,"%.2f" %Accuracy_score_dark_3_std['cam_w_yolo_v5'], ')\\',"\\")
print('&&Y8&',"%.2f" %Accuracy_score_original_mean['cam_w_yolo_FT'],'(' ,"%.2f" %Accuracy_score_original_std['cam_w_yolo_FT'], ')&',"%.2f" %Accuracy_score_bright_1_mean['cam_w_yolo_FT'],'(' ,"%.2f" %Accuracy_score_bright_1_std['cam_w_yolo_FT'], ')&',"%.2f" %Accuracy_score_bright_2_mean['cam_w_yolo_FT'],'(' ,"%.2f" %Accuracy_score_bright_2_std['cam_w_yolo_FT'], ')&',"%.2f" %Accuracy_score_bright_3_mean['cam_w_yolo_FT'],'(' ,"%.2f" %Accuracy_score_bright_3_std['cam_w_yolo_FT'], ')&',"%.2f" %Accuracy_score_dark_1_mean['cam_w_yolo_FT'],'(' ,"%.2f" %Accuracy_score_dark_1_std['cam_w_yolo_FT'], ')&',"%.2f" %Accuracy_score_dark_2_mean['cam_w_yolo_FT'],'(' ,"%.2f" %Accuracy_score_dark_2_std['cam_w_yolo_FT'], ')&',"%.2f" %Accuracy_score_dark_3_mean['cam_w_yolo_FT'],'(' ,"%.2f" %Accuracy_score_dark_3_std['cam_w_yolo_FT'], ')\\',"\\")
print('----------------------------------')
print("%.2f" %Accuracy_score_original_mean['fusion'],'(' ,"%.2f" %Accuracy_score_original_std['fusion'], ')&',"%.2f" %Accuracy_score_bright_1_mean['fusion'],'(' ,"%.2f" %Accuracy_score_bright_1_std['fusion'], ')&',"%.2f" %Accuracy_score_bright_2_mean['fusion'],'(' ,"%.2f" %Accuracy_score_bright_2_std['fusion'], ')&',"%.2f" %Accuracy_score_bright_3_mean['fusion'],'(' ,"%.2f" %Accuracy_score_bright_3_std['fusion'], ')&',"%.2f" %Accuracy_score_dark_1_mean['fusion'],'(' ,"%.2f" %Accuracy_score_dark_1_std['fusion'], ')&',"%.2f" %Accuracy_score_dark_2_mean['fusion'],'(' ,"%.2f" %Accuracy_score_dark_2_std['fusion'], ')&',"%.2f" %Accuracy_score_dark_3_mean['fusion'],'(' ,"%.2f" %Accuracy_score_dark_3_std['fusion'], ')\\',"\\")

67.95 ( 11.02 )& 70.32 ( 8.07 )& 70.94 ( 6.69 )& 70.35 ( 7.53 )& 67.26 ( 11.58 )& 66.01 ( 11.26 )& 69.61 ( 8.04 )\ \
&&Y8& 63.51 ( 15.39 )& 68.03 ( 13.00 )& 74.55 ( 5.88 )& 76.20 ( 6.20 )& 62.60 ( 16.55 )& 63.02 ( 16.70 )& 62.14 ( 18.67 )\ \
----------------------------------
68.49 ( 11.52 )& 65.87 ( 12.49 )& 64.01 ( 13.72 )& 64.53 ( 14.67 )& 67.97 ( 12.61 )& 68.65 ( 12.12 )& 66.92 ( 10.94 )\ \
&&Y8& 70.89 ( 11.13 )& 72.49 ( 10.05 )& 72.69 ( 9.09 )& 73.59 ( 7.99 )& 67.92 ( 13.85 )& 65.19 ( 15.54 )& 64.73 ( 17.19 )\ \
----------------------------------
64.42 ( 15.08 )& 65.26 ( 14.62 )& 66.85 ( 13.16 )& 66.81 ( 13.34 )& 64.17 ( 15.57 )& 63.93 ( 15.73 )& 64.04 ( 17.09 )\ \
